# 股票消息面

## Step 1：輸入需要的套件

In [1]:
import requests
from pyquery import PyQuery as pq
from selenium import webdriver
from time import sleep
import tkinter as tk
from tkinter import ttk

## Step 2：建模

In [2]:
#Button函式
def searchSource():
    if (selectCombo.get()) == "鉅亨網":
        #selenium設定
        driver = webdriver.Chrome("/Users/shijunliao/Desktop/Python/大四下/程式設計概論/chromedriver")
        driver.get("https://news.cnyes.com/news/cat/headline?exp=a")
        html = driver.find_element_by_css_selector("*").get_attribute("outerHTML") 
        doc = pq(html)
        
        #頁面滾動
        for i in range(4):
            height = driver.find_element_by_css_selector("div._2bFl.theme-list").size["height"]
            driver.execute_script("window.scrollTo(0, {})".format(height))
            if i%2 == 0:
                driver.execute_script("window.scrollTo(0, -100)")
            sleep(0.5)
        
        #爬取資料
        dataList = []
        titleList = []
        doc.make_links_absolute(base_url=driver.current_url)
        for eachDoc in doc("._2bFl.theme-list > div > a").items():
            #print(eachDoc.attr("href"))
            mainRes = requests.get(eachDoc.attr("href"))
            mainDoc = pq(mainRes.text)
            for eachItem in mainDoc.items():
                titleList.append(eachItem("h1").text())
                #print(eachItem("h1").text())
                #print(eachItem("._1UuP").text())
                dataList.append({eachItem("h1").text(): eachItem("._1UuP").text()})
        
        #Button函式
        def search():
            result = dataList[titleList.index(news_combo.get())][news_combo.get()]
            result_label.configure(text=result, justify="left", wraplength=1000)
        
        frameNews = tk.Frame(window)
        frameNews.pack(side=tk.TOP)

        news_label = tk.Label(frameNews, text='新聞：')
        news_label.pack(side=tk.LEFT)
        news_combo = ttk.Combobox(frameNews, values=titleList)
        news_combo.pack(side=tk.LEFT)

        result_label = tk.Label(window)
        result_label.pack()

        search_btn = tk.Button(frameNews, text=' 查詢', command=search)
        search_btn.pack()
    
    else:
        res = requests.get("https://www.ptt.cc/bbs/Stock/index.html")
        mainDoc=pq(res.text)
        mainDoc.make_links_absolute(base_url=res.url)

        dataList = []
        titleList = []
        for each in mainDoc("#main-container > div.r-list-container.action-bar-margin.bbs-screen > div > div.title > a").items():
            titleList.append(each.text())
            #print(each.text())
            subRes = requests.get(each.attr("href"))
            subDoc = pq(subRes.text)
            for eachContext in subDoc("#main-content").items():
                dataList.append({each.text(): eachContext.text()})    
                #print(eachContext.text())
                
        #Button函式
        def search():
            result = dataList[titleList.index(news_combo.get())][news_combo.get()]
            result_label.configure(text=result, justify="left", wraplength=1000)
        
        frameNews = tk.Frame(window)
        frameNews.pack(side=tk.TOP)

        news_label = tk.Label(frameNews, text='新聞：')
        news_label.pack(side=tk.LEFT)
        news_combo = ttk.Combobox(frameNews, values=titleList)
        news_combo.pack(side=tk.LEFT)

        result_label = tk.Label(window)
        result_label.pack()

        search_btn = tk.Button(frameNews, text=' 查詢', command=search)
        search_btn.pack()

## Step 3：介面呈現

In [3]:
#初始介面建模
window = tk.Tk()
window.title('Stock News')
window.geometry('1200x600')
window.configure(background='white')

headerLabel = tk.Label(window, text='企業消息查詢')
headerLabel.pack()

frame = tk.Frame(window)
frame.pack(side=tk.TOP)


selectNews = tk.Label(frame, text='消息來源：')
selectNews.pack(side=tk.LEFT)
selectCombo = ttk.Combobox(frame, values=["鉅亨網", "Ptt Stock"])
selectCombo.pack(side=tk.LEFT)


searchBtn = tk.Button(frame, text='搜尋', command=searchSource)
searchBtn.pack(side=tk.LEFT)


window.mainloop()